In [14]:
import requests
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

### 1. Articles scrapping

In [39]:
with open('../.api_auth') as f:
    (api_login, api_pass) = [line.rstrip() for line in f]

ids = []
leads = []
texts = []

date_range = pd.date_range(start="2023-01-01", end="2023-01-31").strftime('%d.%m.%Y').to_list()
for date in tqdm(date_range):
    r1 = requests.get(f'https://api.sta.si/news/sl/{date}', auth=(api_login, api_pass))
    articles_id = eval(r1.content)

    for id in articles_id:
        r2 = requests.get(f'https://api.sta.si/news/sta/{id}', auth=(api_login, api_pass))
        ids.append(id)
        leads.append(r2.json().get('lede', ''))
        texts.append(r2.json().get('text', ''))

dataset = pd.DataFrame({'id': ids, 'lead': leads, 'text': texts})
dataset.to_json(f'../data/raw_articles/01_2023.jsonl', lines=True, orient='records')

100%|██████████| 31/31 [11:34<00:00, 22.41s/it]


### 2. Preparing evaluation dataset

In [47]:
pd.read_json("..\\data\\raw_articles\\all_2023.jsonl", lines=True)

,id,lead,text
0,3123185,,
1,3123019,Slovenska tiskovna agencija d.o.o.\n<pre>\n ...,
2,3122502,Vstopili smo v novo leto 2023. Od starega leta...,Predsednica republike Nataša Pirc Musar je v p...
3,3122717,Slovenska tiskovna agencija (STA) želi vsem na...,
4,3123186,Hrvaška je ob polnoči vstopila v schengensko o...,
...,...,...,...
24897,3165820,Izidi 32. kroga španske nogometne lige in lest...,"<pre>\n* Izidi, 32. krog:\n- petek, 28. april:..."
24898,3165848,Nogometaši Atletica iz Madrida so v 32. krogu ...,"Sredi prvega polčasa je Atletico, za katerega ..."
24899,3165850,Sindikati se morajo tudi v času rekordno nizke...,"Kot je povedal Volf, so se tudi letos pripravi..."
24900,3165853,Igralke v ženski nogometni ligi Triglav Zdravj...,Mura je terensko premoč potrdila v 61. minuti....
